# 10 - 匹配

## 回归到底在做什么？

到目前为止，我们已经看到，当我们进行测试与控制比较时，回归在控制附加变量方面做得非常出色。如果我们有独立性，\\((Y_0, Y_1)\perp T | X\\)，那么回归可以通过控制 X 来识别 ATE。回归的方式有点神奇。为了对它有一些直观的了解，让我们记住所有变量 X 都是虚拟变量的情况。如果是这种情况，回归会将数据划分为虚拟单元格并计算测试和控制之间的平均差异。这种均值差异使 Xs 保持不变，因为我们是在 X dummy 的固定单元格中进行的。就好像我们在做 \\(E[Y|T=1] - E[Y|T=0] | X=x\\)，其中 \\(x\\) 是一个虚拟单元（所有虚拟单元例如设置为 1）。然后回归结合每个单元格中的估计以产生最终的 ATE。这样做的方法是将权重应用到与该组处理的方差成正比的单元格。

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import style
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf

import graphviz as gr

%matplotlib inline

style.use("fivethirtyeight")

举个例子，假设我试图估计一种药物的效果，我有 6 个男人和 4 个女人。 我的反应变量是住院天数，我希望我的药物可以降低住院天数。 对男性而言，真正的因果效应是-3，因此该药物将停留时间缩短了 3 天。 对于女性，它是-2。 更有趣的是，男性更容易受到这种疾病的影响，并且在医院停留的时间更长。 他们也得到了更多的药物。 6 名男性中只有 1 人没有得到药物。 另一方面，女性对这种疾病的抵抗力更强，所以她们在医院的时间更少。 50% 的女性得到了这种药物。

In [2]:
drug_example = pd.DataFrame(dict(
    sex= ["M","M","M","M","M","M", "W","W","W","W"],
    drug=[1,1,1,1,1,0,   1,0,1,0],
    days=[5,5,5,5,5,8,  2,4,2,4]
))

请注意，治疗和对照的简单比较会产生负面影响，即药物似乎不如实际有效。 这是意料之中的，因为我们已经省略了性别混杂因素。 在这种情况下，估计的 ATE 小于真实的 ATE，因为男性服用的药物更多，更容易受到疾病的影响。

In [3]:
drug_example.query("drug==1")["days"].mean() - drug_example.query("drug==0")["days"].mean()

-1.1904761904761898